Previously, I concluded that 'title' feature improves the performance of the model, so it is best to compare the performance of the model with and without the 'keywords' feature using the same set of features, i.e., 'description' plus 'title'. This would allow you to isolate the impact of the 'keywords' feature on the performance of the model, while controlling for the effects of the 'description' and 'title' features. The same applies to the other features.


1. keywords  


2. NER: locations and company names  
 
 
3. type of delivery service (correio Normal, Correio Azul, Correio Registado, Correio Verde, Encomenda Postal, correio prioritário)
or the specific products being delivered (carta, encomenda, )


4. events (Text2Story)  

"CountVectorizer is a good choice when the absolute counts of words are important, such as in sentiment analysis or spam detection. On the other hand, TF-IDFVectorizer is better for tasks where the importance of a word in a document is related to its frequency across all documents, such as text classification or information retrieval."  

In [1]:
# env1
import nltk
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import spacy
import yake
nlp = spacy.load('pt_core_news_lg')
from langdetect import detect
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import hstack
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline, FeatureUnion

In [12]:
# env3
import text2story as t2s
import pandas as pd
t2s.start('pt')
import spacy
spacy.load('pt_core_news_lg')
from text2story.experiments.evaluation import *

error loading _jsonnet (this is expected on Windows), treating D:\MS DATA SCIENCE\NLP TESE\envs\env3\lib\site-packages\text2story\annotators\ALLENNLP\cache\srl-pt_bertimbau-base\config.json as plain json


In [2]:
# Load the data
df = pd.read_csv("D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv")

In [3]:
df.columns

Index(['date', 'title', 'reason', 'description', 'zone', 'narrative_tfidf',
       'narrative_embeddings', 'narrative_bert', 'narrative_tfidf_title',
       'narrative_bert_title', 'narrative_embeddings_title', 'tfidf_title',
       'title_tfidf_nolemma', 'events', 'keywords', 'embeddings_title',
       'tfidf_keywords', 'embeddings_keywords', 'events_clean',
       'results_final', 'events_tfidf', 'events_embeddings', 'orgs', 'locs',
       'entities'],
      dtype='object')

In [ ]:
# # legenda
# reason_dict ={'Mau Serviço Prestado': 0, 'Condições de entrega': 1, 'Atraso de entrega': 2, 
#                'Enganos': 3}

I'm going to use narrative_bert to extract the following features. The package code is interfering with NER results so I'm going to replace them with ''.

## some extra processing

In [16]:
# Define regular expression pattern to remove
pattern = r'_'
# Use the 'replace()' method with the regular expression pattern to remove the string pattern from all rows
df['narrative_bert'] = df['narrative_bert'].replace(to_replace=pattern, value='', regex=True)
df['narrative_bert'].iloc[6306]

'Estou com inúmeros envios para o Brasil, parados em Lisboa desde o início de dezembro de 2017. Faço ENVIO AZUL pela rapidez e sem sucesso. Registro reclamação no site do CTT e é sempre aquela resposta copiada e colada para tudo. E não resolvem nada e nem satisfação dão! Tive um envio no qual a destinatária se mudou de endereço e só chegou ao BR hoje (29/01). Estou pagando por um serviço que não está sendo prestado. Vou listar abaixo os envios com data, código de rastreio e valores. Envio 04/12/2017 Código de Rastreio: LL 0609 9748 1 PT Valor do Envio: € 24,20 Status: Em Lisboa até hoje  Envio 04/12/2017 Código de Rastreio: LL 0609 9749 5 PT Valor do Envio: € 16,50 Status: Em Lisboa até hoje  Envio 18/12/2017 Código de Rastreio: LL 0609 9770 8 PT Valor do Envio: € 5,10 Status: Chegou só hoje no Brasil (29/01) e não vai chegar nas mãos de minha destinatária, pois ela está se mudando de Estado.  Envio 10/01/2018 Código de Rastreio: LL 0609 9809 3 PT Valor do Envio: € 9,00 Status: Em Lisb

In [20]:
# Define regular expression pattern to remove
pattern = r"\b[A-Z]{2}\d{9}[A-Z]{2}\b"
# Use the 'replace()' method with the regular expression pattern to remove the string pattern from all rows
df['narrative_bert'] = df['narrative_bert'].replace(to_replace=pattern, value='', regex=True)
df['narrative_bert'].iloc[0]

'Bom dia venho por este meio apresentar uma reclamação aos CTT. Enviei uma encomenda cujo conteúdo era uma figura de resina artesanal. O envio foi destinado a um amigo de Israel. O envio foi feito no dia 2021-09-09 no posto de ctt da Aboboda. Código:  O envio estava muito bem ambalado e protegido, tinha uma segunda caixa onde estava a figura protegita com muito papel bolha e esponjas. Infelizmente o meu amigo recebeu uma caixa que chegou quase destruida, figura partida, e chegou passado um mês. Mas aqui vão os problemas todos: Levou muito tempo a chegar A caixa chegou destruida A caixa tinha sido aberta e tinha as etiquetas mudadas A caixa tinha um segundo nome de remetente português e um novo remetente em Israel. Pior: A figura chegou danificada e partida, sem concerto. Nas fotos envidas tinha uma etiqueta com novo tracking number alterado e com remetente novo de Braga e destino para outra senhora, que nem sei quem é. Tracking tinha um 6 em vez do 5 anterior no final. Gostaria mesmo d

In [19]:
# Define regular expression pattern to remove
pattern = r" - "
# Use the 'replace()' method with the regular expression pattern to remove the string pattern from all rows
df['narrative_bert'] = df['narrative_bert'].replace(to_replace=pattern, value=' ', regex=True)

## functions to extract features

In [3]:
def extract_orgs(text):
    # locations, company names, not names of people
    doc=nlp(text)
    entities = []
    for named_entity in doc.ents:
        if named_entity.label_ == "ORG":
            entities.append(named_entity.text)
    return entities

def extract_locs(text):
    # locations, company names, not names of people
    doc=nlp(text)
    entities = []
    for named_entity in doc.ents:
        if named_entity.label_ == "LOC":
            entities.append(named_entity.text)
    return entities

def extract_keywords(text):
    language = "pt"
    max_ngram_size = 4
    numOfKeywords = 8
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, top=numOfKeywords, features=None)
    keyword = custom_kw_extractor.extract_keywords(text)
    keyword_list=[k for (k, score) in keyword]
    combined_string = ' '.join(keyword_list)
    return combined_string

def extract_events(text, data):
    # string to datetime
    d = datetime.strptime(data, "%Y-%m-%d %H:%M:%S")
    # convert datetime format into %Y-%m-%d-%H:%M:%S format using strftime
    d=d.strftime("%Y-%m-%d")
    event=[]
    # data['narrative_bert'][i], data['date'][i]
    narrative_doc = t2s.Narrative("pt", text, d)
    extract_element(narrative_doc, "event","allennlp")
    for p in narrative_doc.events:
        event.append(narrative_doc.events[p].text)
    return event

def type_of_mail(text):
    # type of delivery service: 
    doc=nlp(text)
    for chunk in doc.noun_chunks:
        print(chunk.text)

# extract keywords

In [39]:
df['keywords']=df['narrative_bert'].apply(extract_keywords)

In [20]:
import html
def HTMLEntitiesToUnicode(text):
    string=''
    for item in text.split():
        string=string+html.unescape(item)+' '
    result = string.replace('\xa0', ' ')
    return result

# function to tokenize data and remove stopwords
def process_narrative(narrative):
    stopwords_list = nltk.corpus.stopwords.words('portuguese')
    tokens = nltk.word_tokenize(narrative)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in stopwords_list]
    
    # adding line to remove all tokens with numbers and punctuation
    stopwords_punc_and_numbers_removed = [word.lower() for word in stopwords_removed if word.isalpha()]

    return stopwords_punc_and_numbers_removed


# function to concat words (used in function below)
def concat_words(list_of_words):
    # remove any NaN's
    # list_of_words = [i for i in list if i is not np.nan]

    concat_words = ''
    for word in list_of_words:
        concat_words += word + ' '
    return concat_words.strip()


# function to lemmatize words and merge each complaint into a single space-separated string
def make_lemma_and_concat(list_of_words):
    # remove any NaN's
    list_of_words = [i for i in list_of_words if i is not np.nan]
    
    # lemmatize each word
    lemmatized_list = []
    nlp = spacy.load('pt_core_news_md')
    for word in list_of_words:
        doc=nlp(word)
        for token in doc:
            lemmatized_list.append(token.lemma_)    
    
    # make the list into a single string with the words separated by ' '
    concatenated_string = concat_words(lemmatized_list)
    return concatenated_string

def clean_data(texto): #aplica-se a uma so string
    # Remove new line characters
    texto = texto.replace('\n', ' ')
    texto = texto.replace('\r', ' ')
    # Remove multiple spaces
    texto=re.sub(' +', ' ',texto)
    # Remove Emails
    texto = re.sub('\S*@\S*\s?', '',texto)
    # Remove single quotes
    texto = re.sub("\"", '', texto)
    # Remove hyperlinks
    texto = re.sub('http://\S+|https://\S+', '', texto)
    texto = re.sub('http[s]?://\S+', '', texto)
    texto = re.sub(r"http\S+", '', texto)
    texto = re.sub(r'www.\S+', '', texto)
    # Replaces repetitions of exlamation marks
    texto = re.sub(r"(\!)\1+", '!', texto)
    # Replaces repetitions of question marks
    texto = re.sub(r"(\?)\1+", '?', texto)
    # Replaces repetitions of .
    texto = re.sub(r"(\.)\1+", '.', texto)
    # Replaces repetitions of -
    texto = re.sub(r"(\-)\1+", ' ', texto)
    # Replaces repetitions of ,
    texto = re.sub(r"(\,)\1+", ' ', texto)
    return texto

In [41]:
# Preprocessing
df['tfidf_keywords'] = df['keywords'].apply(clean_data).apply(HTMLEntitiesToUnicode).apply(process_narrative).apply(make_lemma_and_concat)

In [42]:
df['tfidf_keywords']

0        apresentar reclamação Caixa figura envio Israe...
1        Lilia pimentel certificar profissional passage...
2        França correio registar França correio encomen...
3        pretender esclarecer situação pretender esclar...
4        papal Centro pois preciso urgentemente pois pr...
                               ...                        
22425    ter chegar Portugal existir qualquer updatar r...
22426    entregar balcao encomenda ctt rebelva ctt venh...
22427    contínuo receber receber encomenda alfândegar ...
22428    processo desembaraço aduaneiro fiz encomenda d...
22429    correspondência extraviar correio destinar adm...
Name: tfidf_keywords, Length: 22430, dtype: object

In [53]:
df['tfidf_keywords']=df['tfidf_keywords'].apply(lambda x: x.lower())

In [55]:
df.to_csv('D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv', index=False, header=True)

In [ ]:
# antes fazer fillna de todas as features 
# Encoding using tf-idf try with different settings
tfidf_vectorizer = TfidfVectorizer(ngram_rangel(1,3),max_df=0.95,min_df=2)
tfidf_vectorizer.fit(df['tfidf_keywords'])  # Fit vectorizer on preprocessed keywords
keywords_tfidf = tfidf_vectorizer.transform(df['tfidf_keywords'])  # Transform preprocessed keywords into tf-idf representation

# Now, you can concatenate keywords_tfidf with the tf-idf encoded 'description' feature for classification
# For example, assuming you have encoded 'description' using tf-idf and stored it in description_tfidf:

# Concatenating features horizontally (adding new columns)
combined_features = np.hstack((description_tfidf.toarray(), keywords_tfidf.toarray()))

# The combined_features array will contain both the tf-idf encoded 'description' and the tf-idf encoded keywords.

# Alternatively, you can concatenate vertically (adding new rows) if you have encoded multiple instances separately.

# Now, you ca

In [39]:
"""
Script used for cleaning corpus in order to train word embeddings.
All emails are mapped to a EMAIL token.
All numbers are mapped to 0 token.
All urls are mapped to URL token.
Different quotes are standardized.
Different hiphen are standardized.
HTML strings are removed.
All text between brackets are removed.
...
"""

from sys import stdout
import argparse
import re
import nltk

sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')

# Punctuation list
punctuations = re.escape('!"#%\'()*+,./:;<=>?@[\\]^_`{|}~')

# ##### #
# Regex #
# ##### #
re_remove_brackets = re.compile(r'\{.*\}')
re_remove_html = re.compile(r'<(\/|\\)?.+?>', re.UNICODE)
re_transform_numbers = re.compile(r'\d', re.UNICODE)
re_transform_emails = re.compile(r'[^\s]+@[^\s]+', re.UNICODE)
re_transform_url = re.compile(r'(http|https)://[^\s]+', re.UNICODE)
# Different quotes are used.
re_quotes_1 = re.compile(r"(?u)(^|\W)[‘’′`']", re.UNICODE)
re_quotes_2 = re.compile(r"(?u)[‘’`′'](\W|$)", re.UNICODE)
re_quotes_3 = re.compile(r'(?u)[‘’`′“”]', re.UNICODE)
re_dots = re.compile(r'(?<!\.)\.\.(?!\.)', re.UNICODE)
re_punctuation = re.compile(r'([,";:]){2},', re.UNICODE)
re_hiphen = re.compile(r' -(?=[^\W\d_])', re.UNICODE)
re_tree_dots = re.compile(u'…', re.UNICODE)
# Differents punctuation patterns are used.
re_punkts = re.compile(r'(\w+)([%s])([ %s])' %
                       (punctuations, punctuations), re.UNICODE)
re_punkts_b = re.compile(r'([ %s])([%s])(\w+)' %
                         (punctuations, punctuations), re.UNICODE)
re_punkts_c = re.compile(r'(\w+)([%s])$' % (punctuations), re.UNICODE)
re_changehyphen = re.compile(u'–')
re_doublequotes_1 = re.compile(r'(\"\")')
re_doublequotes_2 = re.compile(r'(\'\')')
re_trim = re.compile(r' +', re.UNICODE)


def clean_text(text):
    """Apply all regex above to a given string."""
    text = text.lower()
    text = text.replace('\xa0', ' ')
    text = re_tree_dots.sub('...', text)
    text = re.sub('\.\.\.', '', text)
    text = re_remove_brackets.sub('', text)
    text = re_changehyphen.sub('-', text)
    text = re_remove_html.sub(' ', text)
    text = re_transform_numbers.sub('0', text)
    text = re_transform_url.sub('URL', text)
    text = re_transform_emails.sub('EMAIL', text)
    text = re_quotes_1.sub(r'\1"', text)
    text = re_quotes_2.sub(r'"\1', text)
    text = re_quotes_3.sub('"', text)
    text = re.sub('"', '', text)
    text = re_dots.sub('.', text)
    text = re_punctuation.sub(r'\1', text)
    text = re_hiphen.sub(' - ', text)
    text = re_punkts.sub(r'\1 \2 \3', text)
    text = re_punkts_b.sub(r'\1 \2 \3', text)
    text = re_punkts_c.sub(r'\1 \2', text)
    text = re_doublequotes_1.sub('\"', text)
    text = re_doublequotes_2.sub('\'', text)
    text = re_trim.sub(' ', text)
    return text.strip()

In [65]:
df['embeddings_keywords']=df['keywords'].apply(clean_text)

In [68]:
df.to_csv('D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv', index=False, header=True)

In [ ]:
###EMBEDDINGS
# Preprocessing
# # antes fazer fillna de todas as features 

df['keywords_embeddings']

# Integration with Description Feature
combined_features = np.concatenate((description_embeddings, keyword_embeddings), axis=1)

# split data
# ...

In [13]:
print(df['embeddings_keywords'].isna().sum())
print(df['tfidf_keywords'].isna().sum())

0
0


## extract events

1. Preprocessing the Events List:
    * Since the events list contains string elements, you can start by converting the list of strings into a single string. You can use a separator, such as a space or comma, to join the elements together. For example, if you have the events list ['waiting', 'waiting', 'done'], you can convert it to "waiting waiting done".
    * If necessary, apply any additional preprocessing steps to the events string, such as removing punctuation or converting to lowercase.  


2. Encoding the Events String:
    * Use a count vectorizer, which represents the events as frequency counts of each event. Each unique event will be a separate feature in the resulting encoded representation.  
   ex:  Row 1: ['waiting', 'waiting', 'done']
        Row 2: ['in progress', 'in progress', 'waiting']

        Step 1: Preprocessing the Events List:    
        We convert each row's events list into a single string representation:  
        Row 1: "waiting waiting done"  
        Row 2: "in progress in progress waiting"  
        
        Step 2: Countvectorizer  
         we have three unique events across both rows: "waiting", "done", and "in progress". Let's break down the counts for
         each event: Row 1: [2, 1, 0]  
         "waiting" appears twice  
         "done" appears once  
         "in progress" does not appear in this row  
        
    * or TF-IDF

In [121]:
df = pd.read_csv("D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv")

In [ ]:
# import json
# for i in range(len(df)):
#     text,data=df['narrative_bert'].iloc[i],df['date'].iloc[i]
#     events=extract_events(text,data)
#     with open(f'D:\\MS DATA SCIENCE\\NLP TESE\\events\\{i}.txt', 'w',encoding='utf-8') as convert_file:
#         convert_file.write(json.dumps(events))
#     print(f'Finished line number {i}')

In [57]:
# #para ler o output
# import json
# outputs=[]
# for i in range(len(df)):
#     with open(f'D:\\MS DATA SCIENCE\\NLP TESE\\events\\{i}.txt', 'r',encoding='utf-8') as fd:
#         output=json.load(fd)
#         outputs.append(output)

In [97]:
def format_events(text):
    # Remove the brackets and single quotes from the string representation of the list
    text = text.replace("[", "").replace("]", "").replace("'", "")
    # Split the string into individual event strings
    event_strings = text.split(", ")
    # Join the event strings into a single string
    events_string = " ".join(event_strings)
    return events_string

In [101]:
df['events_clean']=df['events'].apply(format_events).apply(lambda x: x.lower())

In [16]:
print(df['events'].isna().sum())
print(df['events_clean'].isna().sum())

0
8


what happens is that when I extracted events, some texts had no events, therefore the output was '[]'  
DROP NAN VALUES BEFORE FITTING MODEL

In [35]:
df['events_tfidf'] = df['events_clean'].fillna('').apply(clean_data).apply(HTMLEntitiesToUnicode).apply(process_narrative).apply(make_lemma_and_concat)

In [40]:
df['events_embeddings']=df['events_clean'].fillna('').apply(clean_text)

In [45]:
print(df['events_tfidf'].isna().sum())
print(df['events_embeddings'].isna().sum())

23
8


In [42]:
df.to_csv('D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv', index=False, header=True)

In [ ]:
# antes fazer fillna de todas as features 
vectorizer = CountVectorizer()
events_encoded = vectorizer.fit_transform(events_strings)
events_encoded_array = events_encoded.toarray()

## extract locs and orgs

In [44]:
# Function to extract organizations and locations
def format_entities(row):
    org = []
    loc = []
    current_org = None
    current_loc = None

    row_list = ast.literal_eval(row)
    for t in row_list:
        if t[1] == 'B-ORGANIZACAO':
            if current_org:
                org.append(current_org)
            current_org = t[0]
        elif t[1] == 'I-ORGANIZACAO':
            current_org = current_org + ' ' + t[0] if current_org else t[0]
        elif t[1] == 'B-LOCAL':
            if current_loc:
                loc.append(current_loc)
            current_loc = t[0]
        elif t[1] == 'I-LOCAL':
            current_loc = current_loc + ' ' + t[0] if current_loc else t[0]

    if current_org:
        org.append(current_org)
    if current_loc:
        loc.append(current_loc)

    return pd.Series({'orgs': ', '.join(org) if org else None, 'locs': ', '.join(loc) if loc else None})

In [45]:
x=df['results_final'][3]
print(x)
y=format_entities(x)
print(y)

[('El', 'B-LOCAL'), ('Corte', 'I-LOCAL'), ('Inglés', 'I-LOCAL'), ('Lisboa', 'B-LOCAL'), ('Correio', 'B-ORGANIZACAO'), ('Verde', 'I-LOCAL')]
orgs                          Correio
locs    El Corte Inglés, Lisboa Verde
dtype: object


In [46]:
import ast
# Apply the function to create separate columns
df[['orgs', 'locs']] = df['results_final'].apply(format_entities)

In [79]:
print(df['orgs'].isna().sum()/len(df))
print(df['locs'].isna().sum()/len(df))

0.4468568880962996
0.46411056620597413


In [84]:
# Function to extract organizations and locations
def format_entities(row):
    org = []
    loc = []
    current_org = None
    current_loc = None

    row_list = ast.literal_eval(row)
    for t in row_list:
        if t[1] == 'B-ORGANIZACAO':
            if current_org:
                org.append(current_org)
            current_org = t[0]
        elif t[1] == 'I-ORGANIZACAO':
            current_org = current_org + ' ' + t[0] if current_org else t[0]
        elif t[1] == 'B-LOCAL':
            if current_loc:
                loc.append(current_loc)
            current_loc = t[0]
        elif t[1] == 'I-LOCAL':
            current_loc = current_loc + ' ' + t[0] if current_loc else t[0]
    if current_org:
        org.append(current_org)
    if current_loc:
        loc.append(current_loc)
    entities = org + loc
    return ', '.join(entities) if entities else None

In [85]:
df['entities'] = df['results_final'].apply(format_entities)

In [87]:
print(df['entities'].isna().sum()/len(df))

0.22697280427998218


In [90]:
# df.to_csv('D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv', index=False, header=True)

## extract products and type of mail  

1. type of mail:
    * correio normal/ azul/ registado/ prioritário/ expresso/ verde
    * encomenda postal
2. product ?? maybe it's not helpfull
    * encomenda
    * carta(s)
    * postal
    * caixa
    * correio

In [69]:
def find_mail_types(text):
    text=text.lower()
    mail_types = re.findall(r'correio normal|correio azul|CTT expresso|correio registado|estrangeiro|internacional|correio prioritário|correio expresso|correio verde', text, flags=re.IGNORECASE)
    letter_types=re.findall(r'carta registada|internacional|prioritária|expressa', text, flags=re.IGNORECASE)
    mail_types = list(set(mail_types))
    letter_types=list(set(letter_types))
    return mail_types + letter_types

In [70]:
for i in rang


e(100):
    print(find_mail_types(df['narrative_bert'].iloc[i]))
    print(df['narrative_bert'].iloc[i])

[]
Bom dia venho por este meio apresentar uma reclamação aos CTT. Enviei uma encomenda cujo conteúdo era uma figura de resina artesanal. O envio foi destinado a um amigo de Israel. O envio foi feito no dia 2021-09-09 no posto de ctt da Aboboda. Código:  O envio estava muito bem ambalado e protegido, tinha uma segunda caixa onde estava a figura protegita com muito papel bolha e esponjas. Infelizmente o meu amigo recebeu uma caixa que chegou quase destruida, figura partida, e chegou passado um mês. Mas aqui vão os problemas todos: Levou muito tempo a chegar A caixa chegou destruida A caixa tinha sido aberta e tinha as etiquetas mudadas A caixa tinha um segundo nome de remetente português e um novo remetente em Israel. Pior: A figura chegou danificada e partida, sem concerto. Nas fotos envidas tinha uma etiqueta com novo tracking number alterado e com remetente novo de Braga e destino para outra senhora, que nem sei quem é. Tracking tinha um 6 em vez do 5 anterior no final. Gostaria mesmo

In [55]:
find_mail_types(df['narrative_bert'][3])

['Correio Verde', 'correio verde']

In [ ]:
# regex
mail_types = re.findall(r'correio normal|correio azul|CTT expresso|correio registado|estrangeiro|internacional|correio prioritário|correio expresso| correio verde', text, flags=re.IGNORECASE)
print(mail_types)  # Output: ['correio expresso']